In [12]:
# project: p13
# submitter: ccmaloney
# partner: jmaloney3
# hours: ???

In [32]:
import os
import pandas as pd
from pandas import DataFrame, Series
import sqlite3
import matplotlib

In [33]:
# ensures that font.size setting remains permanent
%matplotlib inline 
pd.set_option('display.max_colwidth', None)
matplotlib.rcParams["font.size"] = 13 # don't use value > 13! Otherwise your y-axis tick labels will be different. 

In [34]:
# Returns True if file was downloaded, False if file already exists
def download(url, filename = None):
    if filename == None:
        filename = os.path.basename(url)
    # We do not download again if the file already exists
    if os.path.exists(filename):
        return False
    
    response = requests.get(url)
    response.raise_for_status()

    f = open(filename, "w", encoding="utf-8")
    f.write(response.text)
    f.close()

    return True

In [35]:
download("https://raw.githubusercontent.com/msyamkumar/cs220-f21-projects/main/p13/QSranking.json")

False

In [36]:
rankings = pd.read_json('QSranking.json')

In [37]:
conn = sqlite3.connect('rankings.db')
rankings.to_sql("rankings", conn, if_exists = "replace", index = False)

# remember to do conn.close() at the end of your notebook or your code may crash when we run test.py.

In [57]:
#Q1: In the year 2020, list the top 10 institutions in Canada which have the highest value in the column `international_students`.

pd.read_sql(
"""
SELECT institution_name, international_students
FROM rankings
WHERE country = "Canada"
ORDER BY international_students DESC
LIMIT 10
""", conn)



,institution_name,international_students
0,Mcgill University,96.2
1,Mcgill University,96.2
2,Mcgill University,95.6
3,University Of Toronto,93.0
4,University Of Toronto,92.9
5,University Of Toronto,92.1
6,Simon Fraser University,91.9
7,Simon Fraser University,91.8
8,Simon Fraser University,91.5
9,University Of Waterloo,84.1


In [ ]:
conn.close()